In [ ]:
# do all necessary imports for this chapter
import numpy as np
from scipy.linalg import rq, qr, svd
from scipy.sparse.linalg import eigs, LinearOperator
from ncon import ncon

# [Tangent-space methods for uniform matrix product states](https://doi.org/10.21468/SciPostPhysLectNotes.7)

## 1. Matrix product states in the thermodynamic limit
### 1.1 Normalization
We start by considering a uniform MPS in the thermodynamic limit, which is defined as 
$$\left | \Psi(A) \right \rangle = \sum_{\{i\}} \boldsymbol{v}_L^\dagger \left[ \prod_{m\in\mathbb{Z}} A^{i_m} \right] \boldsymbol{v}_R \left | \{i\} \right \rangle.$$

Here, $\boldsymbol{v}_L^\dagger$ and $\boldsymbol{v}_R$ represent boundary vectors at infinity and the $A^i$ are complex matrices of size $D \times D$ for every entry of the index $i$. This allows for the interpretation of the object $A$ as a three-legged tensor of dimensions $D\times d \times D$, where we will refer to $D$ as the bond dimension and $d$ as the physical dimension. With this object and the diagrammatic language of tensor networks, we can represent the state as

<center><img src="./img/umps.svg" alt="MPSstate"/></center>

Thus, we initialize and represent a uniform MPS state as follows:

In [ ]:
def createMPS(D, d):
    """
    Returns a random complex MPS tensor.
    
        Parameters
        ----------
        D : int
            Bond dimension for MPS.
        d : int
            Physical dimension for MPS.
            
        Returns
        -------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
    """
    
    return A

In [ ]:
d, D = 3, 5
A = createMPS(D, d)

assert A.shape == (D, d, D), "Generated MPS tensor has incorrect shape."
assert A.dtype == 'complex128', "MPS tensor should have complex values"

One of the central objects in any MPS calculation is the transfer matrix, defined in our case as

<center><img src="./img/tm.svg" alt="transfer matrix" style="display=block; margin:auto"/></center>

This object corresponds to an operator acting on the space of $D \times D$ matrices, and can be interpreted as a 4-leg tensor. We will use the following convention for ordering the legs:
1. top left
2. bottom left
3. top right
4. bottom right

The transfer matrix can be shown to be a completely positive map, such that its leading eigenvalue is a positive number. This eigenvalue should be rescaled to one to ensure a proper normalization of the state in the thermodynamic limit. To perform this normalization, we must therefore find the left and right fixed points $l$ and $r$ which correspond to the largest eigenvalues of the eigenvalue equations

<center><img src="./img/fixedPoints.svg" alt="fixed points"/></center>

Normalizing the state then means rescaling the MPS tensor $A \leftarrow A / \sqrt{\lambda}$. Additionally, we may fix the normalization of the eigenvectors by requiring that their trace is equal to one:

<center><img src="./img/traceNorm.svg" alt="norm"/></center>

With these properties in place, the norm of an MPS can be evaluated as

<center><img src="./img/mpsNorm.svg" alt="norm"/></center>

It can be readily seen that the infinite product of transfer matrices reduces to a projector onto the fixed points, so that the norm reduces to the overlap between the boundary vectors and the fixed points. Since there is no effect of the boundary vectors on the bulk properties of the MPS, we can always choose these such that MPS is properly normalized as $ \left \langle \Psi(\bar{A})\middle | \Psi(A) \right \rangle = 1$.

In [ ]:
def createTransfermatrix(A):
    """
    Form the transfermatrix of an MPS.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        E : np.array (D, D, D, D)
            Transfermatrix with 4 legs,
            ordered topLeft-bottomLeft-topRight-bottomRight.
    """
    
    # given as an example
    
    E = ncon((A, np.conj(A)), ([-1, 1, -3], [-2, 1, -4]))
    
    return E

In [ ]:
def normalizeMPS(A):
    """
    Normalize an MPS tensor.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        Anew : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalizing (D ** 2, D ** 2) matrix.
    """

    return Anew

In [ ]:
def leftFixedPoint(A):
    """
    Find left fixed point.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        l : np.array (D, D)
            left fixed point with 2 legs,
            ordered bottom-top.
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalizing (D ** 2, D ** 2) matrix.
    """
    
    return l

In [ ]:
def rightFixedPoint(A):
    """
    Find right fixed point.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        r : np.array (D, D)
            right fixed point with 2 legs,
            ordered top-bottom.
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalizing (D ** 2, D ** 2) matrix.
    """
    
    return r

In [ ]:
def fixedPoints(A):
    """
    Find normalized fixed points.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        l : np.array (D, D)
            left fixed point with 2 legs,
            ordered bottom-top.
        r : np.array (D, D)
            right fixed point with 2 legs,
            ordered top-bottom.
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalizing (D ** 2, D ** 2) matrix
    """
    
    return l, r

In [ ]:
A = normalizeMPS(A)
l, r = fixedPoints(A)

assert np.allclose(l, np.conj(l).T, 1e-12), "left fixed point should be hermitian!"
assert np.allclose(r, np.conj(r).T, 1e-12), "right fixed point should be hermitian!"

assert np.allclose(l, ncon((A, l, np.conj(A)), ([1, 2, -2], [3, 1], [3, 2, -1])), 1e-12), "l should be a left fixed point!"
assert np.allclose(r, ncon((A, r, np.conj(A)), ([-1, 2, 1], [1, 3], [-2, 2, 3])), 1e-12), "r should be a right fixed point!"
assert np.abs(ncon((l, r), ([1, 2], [2, 1])) - 1) < 1e-12, "Left and right fixed points should be trace normalized!"

### 1.2 Gauge freedom
While a given MPS tensor $A$ corresponds to a unique state $\left | \Psi(A) \right \rangle$, the converse is not true, as different tensors may give rise to the same state. This is easily seen by noting that the gauge transform

<center><img src="img/gaugeTransform.svg" alt="gauge transform"></center>

leaves the physical state invariant. We may use this freedom in parametrization to impose canonical forms on the MPS tensor $A$.

We start by considering the *left-orthonormal form* of an MPS, which is defined in terms of a tensor $A_L$ that satisfies the condition

<center><img src="img/leftOrth.svg" alt="left orthonormal"></center>

We can find the gauge transform $L$ that brings $A$ into this form

<center><img src="img/leftGauge.svg" alt="left gauge"></center>

by decomposing the fixed point $l$ as $l = L^\dagger L$, such that

<center><img src="img/leftOrth2.svg" alt="left orthonormal2"></center>

Note that this gauge choice still leaves room for unitary gauge transformations

<center><img src="img/unitaryGauge.svg" alt="unitary gauge"></center>

which can be used to bring the right fixed point $r$ into diagonal form. Similarly, we can find the gauge transform that brings $A$ into *right-orthonormal form*

<center><img src="img/rightGauge.svg" alt="right gauge"></center>

such that

<center><img src="img/rightOrth.svg" alt="right gauge"></center>

and the left fixed point $l$ is diagonal. The routines that bring a given MPS into canonical form by decomposing the corresponding transfer matrix fixed points can be defined as follows:

In [ ]:
def leftOrthonormalize(A, l=None):
    """
    Transform A to left-orthonormal gauge.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        L : np.array (D, D)
            left gauge with 2 legs,
            ordered left-right.
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left orthonormal
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalizing (D ** 2, D ** 2) matrix
    """
    
    return L, Al

In [ ]:
def rightOrthonormalize(A, r=None):
    """
    Transform A to right-orthonormal gauge.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        R : np.array (D, D)
            right gauge with 2 legs,
            ordered left-right.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left orthonormal
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalizing (D ** 2, D ** 2) dmatrix
    """
    
    return R, Ar

In [ ]:
L, Al = leftOrthonormalize(A, l)
R, Ar = rightOrthonormalize(A, r)

assert np.allclose(R @ np.conj(R).T, r), "Right gauge doesn't square to r"
assert np.allclose(np.conj(L).T @ L, l), "Left gauge doesn't sqaure to l"
assert np.allclose(ncon((Ar, np.conj(Ar)), ([-1, 1, 2], [-2, 1, 2])), np.eye(D)), "Ar not in right-orthonormal form"
assert np.allclose(ncon((Al, np.conj(Al)), ([1, 2, -2], [1, 2, -1])), np.eye(D)), "Al not in left-orthonormal form"

Finally, we can define a *mixed gauge* for the uniform MPS by choosing one site, the 'center site', and bringing all tensors to the left of it in the left-orthonormal form and all the tensors to the right of it in the right-orthonormal form. Defining a new tensor $A_C$ on the center site, we obtain the form

<center><img src="img/mixedGauge.svg" alt="right gauge"></center>

By contrast, the original representation using the same tensor at every site is commonly referred to as the *uniform gauge*. The mixed gauge has an intuitive interpretation. Defining $C = LR$, this tensor then implements the gauge transform that maps the left-orthonormal tensor to the right-orthonormal one, thereby defining the center-site tensor $A_C$:

<center><img src="img/mixedGauge2.svg" alt="right gauge"></center>

This relation is called the mixed gauge condition and allows us to freely move the center tensor $A_C$ through the MPS, linking the left- and right orthonormal tensors.

Finally we may bring $C$ into diagonal form by performing a singular value decomposition $C = USV^\dagger$ and absorbing $U$ and $V^\dagger$ into the definition of $A_L$ and $A_R$ using the residual unitary gauge freedom

<center><img src="img/diagC.svg" alt="mixed gauge3"></center>

The mixed canonical form with a diagonal $C$ now allows to straightforwardly write down a Schmidt decomposition of the state across an arbitrary bond in the chain

$$ \left | \Psi(A) \right \rangle = \sum_{i=1}^{D} C_i \left | \Psi^i_L(A_L) \right \rangle \otimes \left | \Psi^i_R(A_R) \right \rangle,$$

where the states $\left | \Psi^i_L(A_L) \right \rangle$ and $\left | \Psi^i_R(A_R) \right \rangle$ are orthogonal states on half the lattice. The diagonal elements $C_i$ are exactly the Schmidt numbers of any bipartition of the MPS, and as such determine its bipartite entanglement entropy

$$ S = -\sum_i C_i^2 \log(C_i^2) .$$

In [ ]:
def mixedCanonical(A):
    """
    Bring MPS tensor into mixed gauge, such that -Al-C- = -C-Ar- = Ac.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ac : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right,
            diagonal.
            
        Complexity
        ----------
        O(D ** 6) algorithm,
            diagonalization of (D ** 2, D ** 2) matrix
    """

    return Al, Ac, Ar, C

In [ ]:
def entanglementSpectrum(A):
    """
    Calculate the entanglement spectrum of an MPS.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
            
        Returns
        -------
        S : np.array (D,)
            Singular values of center matrix,
            representing the entanglement spectrum
        entropy : float
            Entanglement entropy across a leg.
    """
    
    return S, entropy

In [ ]:
# check that gauging is still correct
Al, Ac, Ar, C = mixedCanonical(A)
S, entropy = entanglementSpectrum(A)

assert np.allclose(ncon((Ar, np.conj(Ar)), ([-1, 1, 2], [-2, 1, 2])), np.eye(D)), "Ar not in right-orthonormal form"
assert np.allclose(ncon((Al, np.conj(Al)), ([1, 2, -2], [1, 2, -1])), np.eye(D)), "Al not in left-orthonormal form"
LHS = ncon((Al, C), ([-1, -2, 1], [1, -3]))
RHS = ncon((C, Ar), ([-1, 1], [1, -2, -3]))
assert np.allclose(LHS, RHS) and np.allclose(RHS, Ac), "Something went wrong in gauging the MPS"

### 1.3 Truncation of a uniform MPS

The mixed canonical form also enables efficient truncatation an MPS. The sum in the above Schmidt decomposition can be truncated, giving rise to a new MPS that has a reduced bond dimension for that bond. This truncation is optimal in the sense that the norm between the original and the truncated MPS is maximized. To arrive at a translation invariant truncated MPS, we can truncate the columns of the absorbed isometries $U$ and $V^\dagger$ correspondingly, thereby transforming *every* tensor $A_L$ or $A_R$. The truncated MPS in the mixed gauge is then given by

<center><img src="img/truncMPS.svg" alt="truncated MPS"></center>

We note that the resulting state based on this local truncation is not guaranteed to correspond to the MPS with a lower bond dimension that is globally optimal. This would require a variational optimization of the cost function.

$$ \left | \left | ~\left | \Psi(A) \right \rangle - \left | \Psi(\tilde{A}) \right \rangle ~\right | \right |^2.$$

In [ ]:
def truncateMPS(A, Dtrunc):
    """
    Truncate an MPS to a lower bond dimension.
    
        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        Dtrunc : int
            lower bond dimension
            
        Returns
        -------
        AlTilde : np.array (Dtrunc, d, Dtrunc)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        AcTilde : np.array (Dtrunc, d, Dtrunc)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        ArTilde : np.array (Dtrunc, d, Dtrunc)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        CTilde : np.array (Dtrunc, Dtrunc)
            Center gauge with 2 legs,
            ordered left-right,
            diagonal.
    """
    
    return AlTilde, AcTilde, ArTilde, CTilde

In [ ]:
Dtrunc = 3
AlTilde, AcTilde, ArTilde, CTilde = truncateMPS(A, Dtrunc)
assert AlTilde.shape[0] == Dtrunc and AlTilde.shape[2] == Dtrunc, "Something went wrong in truncating the MPS"

### 1.4 Algorithms for finding canonical forms
The success of using MPS for describing physical systems stems from the fact that they provide efficient approximations to a large class of physically relevant states. In one dimension, they have been shown to approximate low-energy states of gapped systems arbitrarily well at only a polynomial cost in the bond dimension $D$. This means that in principle we can push MPS results for these systems to arbitrary precision as long as we increase the bond dimension enough. However, increasing the bond dimension comes at a numerical cost, as the complexity of any MPS algorithm scales with $D$. As opposed to the naive routines given above, it is possible to ensure that the complexity of all MPS algorithms scales as $O(D^3)$, so long as we are a bit careful when implementing them.

As a first example, we can refrain from explicitly contructing the matrices that are used in the eigenvalue problems, and instead pass a function that implements the action of the corresponding operator on a vector to the eigenvalue solver. We demonstrate this for the problem of normalizing an MPS, where instead of explicitly constructing the transfer matrix we now define a function handle which implements its action on the right and left fixed points using an optimal contraction sequence:

In [ ]:
def normalizeMPS(A):
    """
    Normalize an MPS tensor.

        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Returns
        -------
        Anew : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Complexity
        ----------
        O(D ** 3) algorithm,
            D ** 3 contraction for transfer matrix handle.
    """

    return Anew

In [ ]:
def leftFixedPoint(A):
    """
    Find left fixed point.

        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Returns
        -------
        l : np.array (D, D)
            left fixed point with 2 legs,
            ordered bottom-top.

        Complexity
        ----------
        O(D ** 3) algorithm,
            D ** 3 contraction for transfer matrix handle.
    """

    return l

In [ ]:
def rightFixedPoint(A):
    """
    Find right fixed point.

        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Returns
        -------
        r : np.array (D, D)
            right fixed point with 2 legs,
            ordered top-bottom.

        Complexity
        ----------
        O(D ** 3) algorithm,
            D ** 3 contraction for transfer matrix handle.
    """

    return r

In [ ]:
def fixedPoints(A):
    """
    Find normalized fixed points.

        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Returns
        -------
        l : np.array (D, D)
            left fixed point with 2 legs,
            ordered bottom-top.
        r : np.array (D, D)
            right fixed point with 2 legs,
            ordered top-bottom.

        Complexity
        ----------
        O(D ** 3) algorithm,
            D ** 3 contraction for transfer matrix handle.
    """

    return l, r

In [ ]:
A = createMPS(D, d)
A = normalizeMPS(A)
l, r = fixedPoints(A)

assert np.allclose(l, np.conj(l).T, 1e-12), "left fixed point should be hermitian!"
assert np.allclose(r, np.conj(r).T, 1e-12), "right fixed point should be hermitian!"

assert np.allclose(l, ncon((A, l, np.conj(A)), ([1, 2, -2], [3, 1], [3, 2, -1])), 1e-12), "l should be a left fixed point!"
assert np.allclose(r, ncon((A, r, np.conj(A)), ([-1, 2, 1], [1, 3], [-2, 2, 3])), 1e-12), "r should be a right fixed point!"
assert abs(ncon((l, r), ([1, 2], [2, 1])) - 1) < 1e-12, "Left and right fixed points should be trace normalized!"

We can similarly improve both the efficiency and accuracy of the routines bringing a given MPS into its mixed canonical form. While plugging in the more efficient ways of finding the left and right fixed point into the above `mixedCanonical` routine would reduce its complexity to $O(D^3)$, this algorithm would still be suboptimal in terms of numerical accuracy. This arises from the fact that, while $l$ and $r$ are theoretically known to be positive hermitian matrices, at least one of them will nevertheless have small eigenvalues, say of order $\eta$, if the MPS is supposed to provide a good approximation to an actual state. In practice, $l$ and $r$ are determined using an iterative eigensolver and will only be accurate up to a specified tolerance $\epsilon$. Upon taking the 'square roots' $L$ and $R$, the numerical precision will then decrease to $\text{min}(\sqrt{\epsilon}, \epsilon/\sqrt{\eta})$. Furthermore, gauge transforming $A$ with $L$ or $R$ requires the potentially ill-conditioned inversions of $L$ and $R$, and will typically yield $A_L$ and $A_R$ which violate the orthonormalization condition in the same order $\epsilon/\sqrt{\eta}$. We can circumvent both these probelems by resorting to so-called *single-layer algorithms*. These are algorithms that only work on the level of the MPS tensors in the ket layer, and never consider operations for which contractions with the bra layer are needed. We now demonstrate such a single-layer algorithm for finding canonical forms.

Suppose we are given an MPS tensor $A$, then from the above discussion we know that bringing it into left canonical form means finding a left-orthonormal tensor $A_L$ and a matrix $L$ such that $L A=A_L L$. The idea is then to solve this equation iteratively, where in every iteration

1. we start from a matrix $L^{i}$
2. we construct the tensor $L^{i}A$
3. we take a QR decomposition to obtain $A_L^{i+1} L^{i+1} = L^{i}A$, and
4. we take $L^{i+1}$ to the next iteration

The QR decomposition is represented diagrammatically as

<center><img src="img/qrStep.svg" alt="QR step"></center>

This iterative procedure is bound to converge to a fixed point for which $L^{(i+1)}=L^{(i)}=L$ and $A_L$ is left orthonormal by construction:

<center><img src="img/qrConv.svg" alt="QR convergence"></center>

A similar procedure can be used to find a right-orthonormal tensor $A_R$ and a matrix $R$ such that $A R = R A_R$. It is important to note that the convergence of this procedure relies on the fact that the QR decomposition is unique, which is not actually the case in general. However, it can be made unique by imposing that the diagonal elements of the triangular matrix $R$ must be positive. This extra condition is imposed in the routines `qrPos` and `rqPos` defined below.

In [ ]:
def rqPos(A):
    """
    Do a RQ decomposition with positive diagonal elements for R.
    
        Parameters
        ----------
        A : np.array (M, N)
            Matrix to decompose.
            
        Returns
        -------
        R : np.array (M, M)
            Upper triangular matrix,
            positive diagonal elements.
        Q : np.array (M, N)
            Orthogonal matrix.
            
        Complexity
        ----------
        ~O(max(M, N) ** 3) algorithm.
    """

    M, N = A.shape
    
    # LQ decomposition: scipy conventions: Q.shape = (N, N), L.shape = (M, N)
    R, Q = rq(A)

    # Throw out zeros under diagonal: Q.shape = (M, N), L.shape = (M, M)
    Q = Q[-M:, :]
    R = R[:, -M:]

    # Extract signs and multiply with signs on diagonal
    diagSigns = np.diag(np.sign(np.diag(R)))
    Q = np.dot(diagSigns, Q)
    R = np.dot(R, diagSigns)

    return R, Q


def rightOrthonormalize(A, R0=None, tol=1e-14, maxIter=1e5):
    """
    Transform A to right-orthonormal gauge.

        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        R0 : np.array (D, D), optional
            Right gauge matrix,
            initial guess.
        tol : float, optional
            convergence criterium,
            norm(R - Rnew) < tol.
        maxIter : int
            maximum amount of iterations.

        Returns
        -------
        R : np.array (D, D)
            right gauge with 2 legs,
            ordered left-right.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right-orthonormal
    """

    return R, Ar

In [ ]:
def qrPos(A):
    """
    Do a QR decomposition with positive diagonal elements for R.
    
        Parameters
        ----------
        A : np.array (M, N)
            Matrix to decompose.
            
        Returns
        -------
        Q : np.array (M, N)
            Orthogonal matrix.
        R : np.array (N, N)
            Upper triangular matrix,
            positive diagonal elements.
            
        Complexity
        ----------
        ~O(max(M, N) ** 3) algorithm.
    """
    
    M, N = A.shape
    
    # QR decomposition, scipy conventions: Q.shape = (M, M), R.shape = (M, N)
    Q, R = qr(A)
    
    # Throw out zeros under diagonal: Q.shape = (M, N), R.shape = (N, N)
    Q = Q[:, :N]
    R = R[:N, :]

    # extract signs and multiply with signs on diagonal
    diagSigns = np.diag(np.sign(np.diag(R)))
    Q = np.dot(Q, diagSigns)
    R = np.dot(diagSigns, R)
    
    return Q, R


def leftOrthonormalize(A, L0=None, tol=1e-14, maxIter=1e5):
    """
    Transform A to left-orthonormal gauge.

        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        L0 : np.array (D, D), optional
            Left gauge matrix,
            initial guess.
        tol : float, optional
            convergence criterium,
            norm(R - Rnew) < tol.
        maxIter : int
            maximum amount of iterations.

        Returns
        -------
        L : np.array (D, D)
            left gauge with 2 legs,
            ordered left-right.
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left-orthonormal
    """

    return L, Al

In [ ]:
def mixedCanonical(A, L0=None, R0=None, tol=1e-14, maxIter=1e5):
    """
    Bring MPS tensor into mixed gauge, such that -Al-C- = -C-Ar- = Ac.

        Parameters
        ----------
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.

        Returns
        -------
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ac : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        C : np.array (D, D)
            Center gauge with 2 legs,
            ordered left-right,
            diagonal.

        Complexity
        ----------
        O(D ** 3) algorithm.
    """

    return Al, Ac, Ar, C

In [ ]:
Al, Ac, Ar, C = mixedCanonical(A)
S, entropy = entanglementSpectrum(A)

assert np.allclose(ncon((Ar, np.conj(Ar)), ([-1, 1, 2], [-2, 1, 2])), np.eye(D)), "Ar not in right-orthonormal form"
assert np.allclose(ncon((Al, np.conj(Al)), ([1, 2, -2], [1, 2, -1])), np.eye(D)), "Al not in left-orthonormal form"
LHS = ncon((Al, C), ([-1, -2, 1], [1, -3]))
RHS = ncon((C, Ar), ([-1, 1], [1, -2, -3]))
assert np.allclose(LHS, RHS) and np.allclose(RHS, Ac), "Something went wrong in gauging the MPS"

#### Note:
For the remainder of this tutorial, keep in mind that you should always aim to reduce the complexity of your algorithm to $O(D^3)$ in order to keep the computational cost tractable.

### 1.5 Computing expectation values 
Now that we have seen the different ways to parametrize a given MPS, namely the uniform gauge and the mixed gauge, we wish to use these to compute expectation values of an extensive operator:
$$ O = \frac{1}{\mathbb{Z}} \sum_{n \in \mathbb{Z}} O_n. $$

If we assume that each $O_n$ acts on a single site and we are working with a properly normalized MPS, translation invariance dictates that the expectation value of $O$ is given by the contraction

<center><img src="img/expVal.svg" alt="Expectation value"></center>

In the uniform gauge, we can use the fixed points of the transfer matrix to contract everything to the left and to the right of the operator, such that we are left with the contraction

<center><img src="img/expVal2.svg" alt="Expectation value 2"></center>

In the mixed gauge, we can locate the center site where the operator is acting, and then contract everything to the left and right to the identity to arrive at the particularly simple expression

<center><img src="img/expVal3.svg" alt="Expectation value 3"></center>

In [ ]:
def expVal1Uniform(O, A, l=None, r=None):
    """
    Calculate the expectation value of a 1-site operator in uniform gauge.

        Parameters
        ----------
        O : np.array (d, d)
            single-site operator.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.

        Returns
        -------
        o : complex float
            expectation value of O.
    """

    # given as an example:

    # calculate fixed points if not given
    if l is None or r is None:
        l, r = fixedPoints(A)

    # contract expectation value network
    o = ncon((l, r, A, np.conj(A), O), ([4, 1], [3, 6], [1, 2, 3], [4, 5, 6], [2, 5]))

    return o

In [ ]:
def expVal1Mixed(O, Ac):
    """
    Calculate the expectation value of a 1-site operator in mixed gauge.

        Parameters
        ----------
        O : np.array (d, d)
            single-site operator.
        Ac : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauged.

        Returns
        -------
        o : complex float
            expectation value of O.
    """

    return o

In [ ]:
O = np.random.rand(d,d) + 1.0j * np.random.rand(d,d)
A = createMPS(D, d)
A = normalizeMPS(A)
Al, Ac, Ar, C = mixedCanonical(A)
expVal = expVal1Uniform(O, A)
expValMix = expVal1Mixed(O, Ac)
diff = abs(expVal - expValMix)
assert diff < 1e-14, "different gauges give different values?"

This procedure can be readily generalized to operators that act on multiple sites. In particular, a two-site operator such as a Hamiltonian term $h$ can be evaluated as

<center><img src="img/expValHam.svg" alt="Expectation value Hamiltonian"></center>

In [ ]:
def expVal2Uniform(O, A, l=None, r=None):
    """
    Calculate the expectation value of a 2-site operator in uniform gauge.

        Parameters
        ----------
        O : np.array (d, d, d, d)
            two-site operator,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        A : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right.
        l : np.array (D, D), optional
            left fixed point of transfermatrix,
            normalized.
        r : np.array (D, D), optional
            right fixed point of transfermatrix,
            normalized.

        Returns
        -------
        o : complex float
            expectation value of O.
    """

    return o

In [ ]:
def expVal2Mixed(O, Ac, Ar):
    """
    Calculate the expectation value of a 2-site operator in mixed gauge.

        Parameters
        ----------
        O : np.array (d, d, d, d)
            two-site operator,
            ordered topLeft-topRight-bottomLeft-bottomRight.
        Ac : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauged.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right gauged.

        Returns
        -------
        o : complex float
            expectation value of O.
    """

    return o

In [ ]:
O2 = np.random.rand(d, d, d, d) + 1.0j * np.random.rand(d, d, d, d)

expVal = expVal2Uniform(O2, A)
expValGauge = expVal2Mixed(O2, Ac, Ar)
expValGauge2 = expVal2Mixed(O2, Al, Ac)

diff1 = abs(expVal - expValGauge)
diff2 = abs(expVal - expValGauge2)
assert diff1 < 1e-12 and diff2 < 1e-12, "different gauges give different values?"